<a href="https://colab.research.google.com/github/leoalfonso/AI4water/blob/main/Exercise_1_Water_allocation_farmer_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module: Artificial Intelligence for Water Systems
## Topic: Optimisation

Solution for Exercise 1 (water allocation problem) using Python  
By: L. Alfonso

## Problem Statement

A farmer has two fields: Field 1 with **strawberries** and Field 2 with **cucumbers**. Both crops can grow without additional water, but productivity increases linearly with irrigation. The productivity and profitability details are:

### Field 1 – Strawberries
- Fixed production (without water): **2000 kg**
- Increase in production per m³ of water: **5 kg**
- Profit per kg: **$5**

### Field 2 – Cucumbers
- Fixed production (without water): **3000 kg**
- Increase in production per m³ of water: **25 kg**
- Profit per kg: **$4**

### Constraints
- The total available water for irrigation is **500 m³**.
- The processing and packaging facility has a maximum capacity of **9000 kg**.

The farmer’s total **profit** is the total weight processed and packaged multiplied by the respective profit per kg.

---

## Problem Summary (for modelling)

Let:
- \( $x_1$ \): Water allocated to **strawberries** (in m³)
- \( $x_2$ \): Water allocated to **cucumbers** (in m³)

### Objective Function (maximize profit):

$\text{Profit} = 5 \cdot (2000 + 5x_1) + 4 \cdot (3000 + 25x_2) = 22000 + 25x_1 + 100x_2$

### Constraints:
1. **Water availability**:  
$x_1 + x_2 \leq 500$

2. **Processing capacity**:  
$(2000 + 5x_1) + (3000 + 25x_2) \leq 9000 \quad \Rightarrow \quad 5x_1 + 25x_2 \leq 4000$

3. **Non-negativity**:  
$x_1 \geq 0,\quad x_2 \geq 0$

In [ ]:
from scipy.optimize import linprog

# Coefficients for the objective function (to be minimized → we negate for maximization)
# Profit = 5*(2000 + 5x1) + 4*(3000 + 25x2)
#        = 10000 + 25x1 + 12000 + 100x2
#        = 22000 + 25x1 + 100x2 → maximize
# We'll minimize the negative: -25x1 - 100x2

c = [-25, -100]  # Coefficients for x1 and x2 (water to strawberries, cucumbers)

# Constraints:
# 1. x1 + x2 <= 500 (water constraint)
# 2. (2000 + 5x1) + (3000 + 25x2) <= 9000 → 5x1 + 25x2 <= 4000

A = [
    [1, 1],       # x1 + x2 <= 500
    [5, 25]       # 5x1 + 25x2 <= 4000
]

b = [500, 4000]

# Bounds for x1 and x2: both >= 0
x_bounds = (0, None)
bounds = [x_bounds, x_bounds]

# Solve the linear program
result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')

# Output results
if result.success:
    x1, x2 = result.x
    total_profit = 5 * (2000 + 5 * x1) + 4 * (3000 + 25 * x2)
    print("Optimal water allocation:")
    print(f"  Water to strawberries (x1): {x1:.2f} m³")
    print(f"  Water to cucumbers   (x2): {x2:.2f} m³")
    print(f"Total profit: ${total_profit:.2f}")
else:
    print("Optimization failed:", result.message)


## Feasible Region

The **feasible region** is the set of all $(x_1, x_2)$ combinations that satisfy both constraints:

- It is bounded by:
  - The line $(x_1 + x_2 = 500)$ (water limit)
  - The line $(5x_1 + 25x_2 = 4000$) (processing capacity)
  - The axes $(x_1 = 0)$ and $(x_2 = 0$)

This region is a **convex polygon** and contains all valid irrigation strategies. The optimal profit will occur at one of the **vertices** (corner points) of this polygon, which we can identify visually or solve with linear programming.


The following plot shows:

- The two constraint boundaries
- The shaded feasible region
- The optimal point that maximizes profit (as found by linear programming)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import linprog

# Define the lines
x1 = np.linspace(0, 600, 400)

# Constraint 1: x1 + x2 <= 500 → x2 = 500 - x1
x2_water = 500 - x1

# Constraint 2: 5x1 + 25x2 <= 4000 → x2 = (4000 - 5x1)/25
x2_processing = (4000 - 5 * x1) / 25

# Set up the plot
plt.figure(figsize=(8, 6))
plt.plot(x1, x2_water, label='Water limit: $x_1 + x_2 \\leq 500$', color='blue')
plt.plot(x1, x2_processing, label='Processing limit: $5x_1 + 25x_2 \\leq 4000$', color='green')

# Fill feasible region
x_fill = np.linspace(0, 500, 400)
x2_fill = np.minimum(500 - x_fill, (4000 - 5 * x_fill) / 25)
x2_fill = np.maximum(x2_fill, 0)
plt.fill_between(x_fill, x2_fill, color='lightgrey', label='Feasible region')

# Solve the LP to get optimal point
c = [-25, -100]
A = [[1, 1], [5, 25]]
b = [500, 4000]
bounds = [(0, None), (0, None)]

res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')

if res.success:
    opt_x1, opt_x2 = res.x
    plt.plot(opt_x1, opt_x2, 'ro', label='Optimal solution')
    plt.text(opt_x1 + 5, opt_x2, f'({opt_x1:.1f}, {opt_x2:.1f})', color='red')

# Labels and legend
plt.xlabel("Water to strawberries ($x_1$) [m³]")
plt.ylabel("Water to cucumbers ($x_2$) [m³]")
plt.title("Feasible Region and Optimal Irrigation Strategy")
plt.xlim(0, 600)
plt.ylim(0, 300)
plt.grid(True)
plt.legend()
plt.show()

## Using `scipy.optimize.linprog` for Linear Programming

We use the `linprog` function from Python’s **SciPy** library to solve linear programming problems. Linear programming is a method to achieve the **best outcome** (such as maximum profit or minimum cost) in a mathematical model whose requirements are represented by **linear relationships**.

### What is SciPy?

**SciPy** is an open-source Python library used for scientific and technical computing. It builds on **NumPy** and provides additional modules for optimization, integration, interpolation, and more.

The `scipy.optimize` module specifically provides tools for solving optimisation problems, including:

- Linear programming (`linprog`)
- Nonlinear optimisation
- Root finding
- Curve fitting

---

## `linprog` Function

It solves problems of the form:

$
\text{Minimize:} \quad c^T x \\
\text{Subject to:} \quad A_{ub} x \leq b_{ub} \\
\quad A_{eq} x = b_{eq} \\
\quad \text{Bounds on } x
$


Where:
- `c`: Coefficients of the objective function (1D array)
- `A_ub`, `b_ub`: Inequality constraint matrix and vector
- `A_eq`, `b_eq`: Equality constraint matrix and vector (optional)
- `bounds`: Variable bounds, e.g., $( x \geq 0 )$

---

## Important Notes

- `linprog` **minimises** the objective function.  
  To **maximise**, we multiply the coefficients by `-1` and interpret the result accordingly.

- The `method='highs'` argument specifies a modern, efficient solver suitable for most problems.



---



---

# Code to explore the six-hump camellback equation
## (not related to the water allocation problem)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define the function
def func(x, y):
    return (1.036285 + 4 * x**2 - 2.1 * x**4 + (1/3) * x**6 +
            x * y - 4 * y**2 + 4 * y**4)

# Create a grid of x and y values
x = np.linspace(-2, 2, 400)
y = np.linspace(-1, 1, 400)
X, Y = np.meshgrid(x, y)
Z = func(X, Y)

# Plot the 3D surface
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Adjust the viewing angle
ax.view_init(elev=15, azim=-40)  # <--- change these values to rotate the view


# Surface plot
surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none', alpha=0.9)

# Labels and title
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title('3D Plot of z = 1.036285 + 4x² - 2.1x⁴ + (1/3)x⁶ + xy - 4y² + 4y⁴')

# Color bar
fig.colorbar(surf, shrink=0.5, aspect=10)

plt.show()


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Define the function
def func(x, y):
    return (1.036285 + 4 * x**2 - 2.1 * x**4 + (1/3) * x**6 +
            x * y - 4 * y**2 + 4 * y**4)

# Create a grid of x and y values
x = np.linspace(-2, 2, 100) # Reduce points for faster rendering
y = np.linspace(-1, 1, 100) # Reduce points for faster rendering
X, Y = np.meshgrid(x, y)
Z = func(X, Y)

# Create the 3D surface plot using Plotly
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='viridis')])

# Update layout for better readability and title
fig.update_layout(
    title='Interactive 3D Plot of z = 1.036285 + 4x² - 2.1x⁴ + (1/3)x⁶ + xy - 4y² + 4y⁴',
    scene = dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z'
    ),
    width=800 # Adjust the width here
)

# Display the plot
fig.show()